## Analysis of the Quasiparticle Method from an analytical perspective

#### Analysis of the effective one-body matrix

Imports

In [1]:

from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverterOnlynnpp,HardcoreBosonsBasis
from scipy.sparse import lil_matrix
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=0

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{18}$O'

/opt/anaconda3/envs/dft_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


NSM Hamiltonian

In [2]:

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)
print(NSMHamiltonian.basis)
print(NSMHamiltonian.hamiltonian)
print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 24 12
[-11.93178825]
[[1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
  (0, 0)	-9.3151
  (0, 1)	0.6897999999999996
  (0, 2)	0.20328479938540372
  (0, 3)	-0.4062999999999996
  (0, 4)	-0.5082840208321855
  (0, 5)	0.18949464964372487
  (0, 6)	0.5082840208321854
  (0, 7)	0.18949464964372495
  (0, 8)	0.20328479938540372
  (0, 9)	-0

Quasiparticle Basis

In [3]:
QPC=QuasiParticlesConverterOnlynnpp()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

print(QPC.particles2quasiparticles.shape)
print(QPC.quasiparticle_basis)


(6, 14)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


Get the effective Hamiltonian

In [4]:

hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_qr=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_rq=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T


values,psi=eigsh(hamiltonian_qq,k=1)
e0=values[0]

print(values[0])
values,psi=eigsh(hamiltonian_qq+(1/e0)*hamiltonian_qr @ hamiltonian_rq,k=1)

e1=values[0]

values,psi=eigsh(hamiltonian_qq+(1/e1)*hamiltonian_qr @ hamiltonian_rq+(1/e1**2)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rq,k=1)

e2=values[0]
values,psi=eigsh(hamiltonian_qq+(1/e2)*hamiltonian_qr @ hamiltonian_rq+(1/e2**2)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rq+(1/e2**3)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rr @ hamiltonian_rq,k=1)


print(values[0],np.abs((values[0]-egs[0])/egs[0]))

print(hamiltonian_qq)

-11.931788251561525
-11.931788251561523 5.95503977090032e-16
  (0, 0)	-9.3151
  (0, 1)	0.6897999999999996
  (0, 2)	-0.4062999999999996
  (0, 3)	-0.900781889989654
  (0, 4)	-1.9314456782406446
  (0, 5)	0.6017349640876247
  (1, 0)	0.6897999999999996
  (1, 1)	-8.8615
  (1, 2)	0.8598999999999998
  (1, 3)	0.900781889989654
  (1, 4)	1.1336192497488327
  (1, 5)	-1.3995613925794368
  (2, 0)	-0.4062999999999996
  (2, 1)	0.8598999999999998
  (2, 2)	-9.145
  (2, 3)	-0.900781889989654
  (2, 4)	-0.7347060355029268
  (2, 5)	1.7984746068253425
  (3, 0)	-0.900781889989654
  (3, 1)	0.900781889989654
  (3, 2)	-0.900781889989654
  (3, 3)	-8.107099999999999
  (3, 4)	-0.7177133829043456
  (3, 5)	0.7177133829043456
  (4, 0)	-1.9314456782406446
  (4, 1)	1.1336192497488327
  (4, 2)	-0.7347060355029268
  (4, 3)	-0.7177133829043456
  (4, 4)	3.2251000000000003
  (4, 5)	0.9008999999999998
  (5, 0)	0.6017349640876247
  (5, 1)	-1.3995613925794368
  (5, 2)	1.7984746068253425
  (5, 3)	0.7177133829043456
  (5, 4)	0.90

Get the effective coupling

In [5]:
twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
print(twobody_qr)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

print(external_potential_qq)



  (0, 0)	0.20328479938540372
  (0, 1)	-0.5082840208321855
  (0, 2)	0.18949464964372487
  (0, 3)	0.5082840208321854
  (0, 4)	0.18949464964372495
  (0, 5)	0.20328479938540372
  (0, 6)	0.48367995370807726
  (0, 7)	0.48367995370807726
  (1, 0)	0.30763695987708084
  (1, 1)	-0.10165680416643715
  (1, 2)	0.6918637014569979
  (1, 3)	0.10165680416643713
  (1, 4)	0.691863701456998
  (1, 5)	0.30763695987708073
  (1, 6)	0.09673599074161551
  (1, 7)	0.09673599074161551
  (2, 0)	0.10435216049167703
  (2, 1)	0.4066272166657483
  (2, 2)	0.502369051813273
  (2, 3)	-0.4066272166657482
  (2, 4)	0.502369051813273
  (2, 5)	0.104352160491677
  (2, 6)	-0.3869439629664618
  (2, 7)	-0.3869439629664618
  (4, 0)	0.23293633710277395
  (4, 1)	-0.15865000000000007
  (4, 2)	-0.09509586140912163
  (4, 3)	0.15865000000000004
  (4, 4)	-0.09509586140912159
  (4, 5)	0.232936337102774
  (4, 6)	0.17470000000000002
  (4, 7)	0.17470000000000002
  (5, 0)	0.23293633710277395
  (5, 1)	-0.15865000000000007
  (5, 2)	-0.0950958614

Build $H_{qq}$ using qubits

In [6]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.
for i in range(len(QPC.couples)//2): #just n-n 
    for j in range(len(QPC.couples)//2):
        
        h_qq+=hamiltonian_qq[i,j]*QubitBasis.adag_a_matrix(i,j)
        




one_body={}
for i,b in enumerate(QPC.quasiparticle_basis):
    for j,d in enumerate(QPC.quasiparticle_basis):
        idx_b=np.nonzero(b)[0]
        idx_d=np.nonzero(d)[0]
        print(idx_d,idx_b)
        one_body[(idx_b[0],idx_d[0])]=hamiltonian_qq[i,j]
        one_body[(idx_b[0]+len(QPC.couples)//2,idx_d[0]+len(QPC.couples)//2)]=hamiltonian_qq[i,j]

print(one_body.keys())

[0] [0]
[1] [0]
[2] [0]
[3] [0]
[4] [0]
[5] [0]
[0] [1]
[1] [1]
[2] [1]
[3] [1]
[4] [1]
[5] [1]
[0] [2]
[1] [2]
[2] [2]
[3] [2]
[4] [2]
[5] [2]
[0] [3]
[1] [3]
[2] [3]
[3] [3]
[4] [3]
[5] [3]
[0] [4]
[1] [4]
[2] [4]
[3] [4]
[4] [4]
[5] [4]
[0] [5]
[1] [5]
[2] [5]
[3] [5]
[4] [5]
[5] [5]
dict_keys([(0, 0), (6, 6), (0, 1), (6, 7), (0, 2), (6, 8), (0, 3), (6, 9), (0, 4), (6, 10), (0, 5), (6, 11), (1, 0), (7, 6), (1, 1), (7, 7), (1, 2), (7, 8), (1, 3), (7, 9), (1, 4), (7, 10), (1, 5), (7, 11), (2, 0), (8, 6), (2, 1), (8, 7), (2, 2), (8, 8), (2, 3), (8, 9), (2, 4), (8, 10), (2, 5), (8, 11), (3, 0), (9, 6), (3, 1), (9, 7), (3, 2), (9, 8), (3, 3), (9, 9), (3, 4), (9, 10), (3, 5), (9, 11), (4, 0), (10, 6), (4, 1), (10, 7), (4, 2), (10, 8), (4, 3), (10, 9), (4, 4), (10, 10), (4, 5), (10, 11), (5, 0), (11, 6), (5, 1), (11, 7), (5, 2), (11, 8), (5, 3), (11, 9), (5, 4), (11, 10), (5, 5), (11, 11)])


#### Proton-Neutron Case

Imports

In [7]:

from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverter,HardcoreBosonsBasis
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=2

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{20}$O'

NSM Hamiltonian

In [43]:

file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=2

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{20}$Ne'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 24 12
[-41.39649007]
total_m= [-7.65717784e-18]


Quasiparticle Basis

In [44]:
QPC=QuasiParticlesConverterOnlynnpp()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)
for b in QPC.quasiparticle_basis:
    if np.sum(b)!=2:
        print(np.sum(b),'\n')



Get the effective Hamiltonian

In [45]:
twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
print(twobody_qq)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

print(external_potential_qq)

  (0, 0)	-14.51434603174604
  (0, 1)	0.6897999999999995
  (0, 2)	-0.4062999999999996
  (0, 3)	-0.9007818899896537
  (0, 4)	-1.9314456782406437
  (0, 5)	0.6017349640876244
  (0, 6)	0.6897999999999996
  (0, 12)	-0.4062999999999996
  (0, 18)	-0.900781889989654
  (0, 24)	-1.9314456782406446
  (0, 30)	0.6017349640876247
  (1, 0)	0.6897999999999995
  (1, 1)	-9.485076190476194
  (1, 2)	0.8598999999999996
  (1, 3)	0.9007818899896537
  (1, 4)	1.133619249748832
  (1, 5)	-1.3995613925794361
  (1, 7)	0.6897999999999996
  (1, 13)	-0.4062999999999996
  (1, 19)	-0.900781889989654
  (1, 25)	-1.9314456782406446
  (1, 31)	0.6017349640876247
  (2, 0)	-0.4062999999999996
  (2, 1)	0.8598999999999996
  (2, 2)	-8.407544444444449
  :	:
  (33, 33)	-8.029499999999999
  (33, 34)	-0.7177133829043454
  (33, 35)	0.7177133829043454
  (34, 4)	0.6017349640876247
  (34, 10)	-1.3995613925794368
  (34, 16)	1.7984746068253425
  (34, 22)	0.7177133829043456
  (34, 28)	0.9008999999999998
  (34, 30)	-1.9314456782406437
  (34,

In [46]:
hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_qr=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_rq=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
#print(twobody_qr)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

#### twobody terms
twobody_nnpp={}
for i,b in enumerate(QPC.quasiparticle_basis):
    for j,d in enumerate(QPC.quasiparticle_basis):
        idx_a,idx_b=np.nonzero(b)[0]
        idx_c,idx_d=np.nonzero(d)[0]
        if hamiltonian_qq[i,j]-h_qq[i,j]!=0.:
            twobody_nnpp[(idx_a,idx_b,idx_c,idx_d)]=hamiltonian_qq[i,j]-h_qq[i,j]


for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)




value,_=eigsh(h_qq,k=1)
print(h_qq)
print(value)

dict_keys([(0, 0), (6, 6), (0, 1), (6, 7), (0, 2), (6, 8), (0, 3), (6, 9), (0, 4), (6, 10), (0, 5), (6, 11), (1, 0), (7, 6), (1, 1), (7, 7), (1, 2), (7, 8), (1, 3), (7, 9), (1, 4), (7, 10), (1, 5), (7, 11), (2, 0), (8, 6), (2, 1), (8, 7), (2, 2), (8, 8), (2, 3), (8, 9), (2, 4), (8, 10), (2, 5), (8, 11), (3, 0), (9, 6), (3, 1), (9, 7), (3, 2), (9, 8), (3, 3), (9, 9), (3, 4), (9, 10), (3, 5), (9, 11), (4, 0), (10, 6), (4, 1), (10, 7), (4, 2), (10, 8), (4, 3), (10, 9), (4, 4), (10, 10), (4, 5), (10, 11), (5, 0), (11, 6), (5, 1), (11, 7), (5, 2), (11, 8), (5, 3), (11, 9), (5, 4), (11, 10), (5, 5), (11, 11)])
  (0, 0)	-30.21714603174604
  (0, 1)	0.6897999999999995
  (0, 2)	-0.4062999999999996
  (0, 3)	-0.9007818899896537
  (0, 4)	-1.9314456782406437
  (0, 5)	0.6017349640876244
  (0, 6)	0.6897999999999996
  (0, 12)	-0.4062999999999996
  (0, 18)	-0.900781889989654
  (0, 24)	-1.9314456782406446
  (0, 30)	0.6017349640876247
  (1, 0)	0.6897999999999995
  (1, 1)	-25.187876190476196
  (1, 2)	0.859

Get the effective (p-p)--(n-n) interaction

In [47]:




tot_hamiltonian=hamiltonian_qq
values,psi=eigsh(hamiltonian_qq,k=1)
e=values[0]
approximations=[]
nsteps_iteration=30
single_term = hamiltonian_rq  # Start with initial term
for i in trange(nsteps_iteration):
    if i > 0:
        single_term = hamiltonian_rr @ single_term  # Efficient update
    approximations.append(hamiltonian_qr @ single_term)  # Store result

two_body_corrections=[]
for operator in approximations:
    twobody_term={}
    for i,b in enumerate(QPC.quasiparticle_basis):
        for j,d in enumerate(QPC.quasiparticle_basis):
            idx_a,idx_b=np.nonzero(b)[0]
            idx_c,idx_d=np.nonzero(d)[0]
            
            twobody_term[(idx_a,idx_b,idx_c,idx_d)]=operator[i,j]#-external_potential_qq[i,j]
    two_body_corrections.append(twobody_term)


 
history_errors_exact=[]
for i in trange(nsteps_iteration):
    
    tot_hamiltonian=hamiltonian_qq
    for j in range(i):
        tot_hamiltonian=tot_hamiltonian+approximations[j]/e**(j+1)
    values,psi=eigsh(tot_hamiltonian,k=1)
    e=values[0]
    print(e)
    print(np.abs((e-egs[0])/egs[0]),'index=',i)
    history_errors_exact.append(np.abs((e-egs[0])/egs[0]))


100%|██████████| 30/30 [00:00<00:00, 228.42it/s]

-32.20884586
0.22194258973317837 index= 0
-34.319259916768836
0.17096208261604878 index= 1
-36.12155194839493
0.12742476753289578 index= 2
-37.45880480565924
0.09512123515804845 index= 3
-38.41745070899184
0.07196357368356006 index= 4
-39.11324127389526
0.05515561330615432 index= 5
-39.62517873384157
0.04278892571865936 index= 6
-40.00833174512998
0.0335332373591699 index= 7
-40.2988906531142
0.02651431117787152 index= 8
-40.52226146513022
0.0211184234679608 index= 9
-40.69565638978868
0.016929785147308717 index= 10
-40.83170395421679
0.013643333465303477 index= 11
-40.939147469801505
0.011047859419161903 index= 12
-41.02476333038116
0.00897966812010323 index= 13
-41.09322482182349
0.007325868695989939 index= 14
-41.14844489107942
0.005991937506192745 index= 15
-41.19297671445537
0.004916198396478816 index= 16
-41.229272359094736
0.004039417670003243 index= 17
-41.25867331179464
0.0033291895166414317 index= 18
-41.28289905482986
0.002743977026578745 index= 19
-41.30249886774649
0.00227

In [48]:
print(twobody_nnpp.keys())

dict_keys([(0, 6, 0, 6), (0, 6, 0, 7), (0, 6, 0, 9), (0, 6, 0, 10), (0, 6, 0, 11), (0, 7, 0, 6), (0, 7, 0, 7), (0, 7, 0, 8), (0, 7, 0, 9), (0, 7, 0, 10), (0, 7, 0, 11), (0, 8, 0, 7), (0, 8, 0, 8), (0, 8, 0, 9), (0, 8, 0, 10), (0, 8, 0, 11), (0, 9, 0, 6), (0, 9, 0, 7), (0, 9, 0, 8), (0, 9, 0, 9), (0, 9, 0, 10), (0, 9, 0, 11), (0, 10, 0, 6), (0, 10, 0, 7), (0, 10, 0, 8), (0, 10, 0, 9), (0, 10, 0, 10), (0, 10, 0, 11), (0, 11, 0, 6), (0, 11, 0, 7), (0, 11, 0, 8), (0, 11, 0, 9), (0, 11, 0, 10), (0, 11, 0, 11), (1, 6, 1, 6), (1, 6, 1, 7), (1, 6, 1, 9), (1, 6, 1, 10), (1, 6, 1, 11), (1, 7, 1, 6), (1, 7, 1, 7), (1, 7, 1, 8), (1, 7, 1, 9), (1, 7, 1, 10), (1, 7, 1, 11), (1, 8, 1, 7), (1, 8, 1, 8), (1, 8, 1, 9), (1, 8, 1, 10), (1, 8, 1, 11), (1, 9, 1, 6), (1, 9, 1, 7), (1, 9, 1, 8), (1, 9, 1, 9), (1, 9, 1, 10), (1, 9, 1, 11), (1, 10, 1, 6), (1, 10, 1, 7), (1, 10, 1, 8), (1, 10, 1, 9), (1, 10, 1, 10), (1, 10, 1, 11), (1, 11, 1, 6), (1, 11, 1, 7), (1, 11, 1, 8), (1, 11, 1, 9), (1, 11, 1, 10), (1, 1

#### Two-body term (nn)--(nn)

In [34]:
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=4
nparticles_b=0

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{20}$O'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 24 12
[-23.93429271]
total_m= [0.]


Pairing Hamiltonian one-body term

In [35]:
QPC=QuasiParticlesConverterOnlynnpp()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

print(QPC.particles2quasiparticles.shape)
print(QPC.quasiparticle_basis)

print(QPC.rest_basis.shape)
print(QPC.couples)
print(QPC.rest_basis)

external_potential_qubit=np.zeros(len(QPC.couples))

external_potential_qubit=2*SPS.energies


(15, 81)
[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]]
(66, 24)
[[0, 5], [1, 4], [2, 3], [6, 7], [8, 11], [9, 10], [12, 17], [13, 16], [14, 15], [18, 19], [20, 23], [21, 22]]
[[0 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


#### Two-body correction

In [36]:

hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_qr=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_rq=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
#print(twobody_qr)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

#### twobody terms
twobody_nn={}
for i,b in enumerate(QPC.quasiparticle_basis):
    for j,d in enumerate(QPC.quasiparticle_basis):
        idx_a,idx_b=np.nonzero(b)[0]
        idx_c,idx_d=np.nonzero(d)[0]
        if hamiltonian_qq[i,j]-h_qq[i,j]!=0.:
            twobody_nn[(idx_a,idx_b,idx_c,idx_d)]=hamiltonian_qq[i,j]-h_qq[i,j]
            twobody_nn[(idx_a+len(QPC.couples)//2,idx_b+len(QPC.couples)//2,idx_c+len(QPC.couples)//2,idx_d+len(QPC.couples)//2)]=hamiltonian_qq[i,j]-h_qq[i,j]
            

 

h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

value,_=eigsh(h_qq,k=1)
print(h_qq)
print(value)

dict_keys([(0, 0), (6, 6), (0, 1), (6, 7), (0, 2), (6, 8), (0, 3), (6, 9), (0, 4), (6, 10), (0, 5), (6, 11), (1, 0), (7, 6), (1, 1), (7, 7), (1, 2), (7, 8), (1, 3), (7, 9), (1, 4), (7, 10), (1, 5), (7, 11), (2, 0), (8, 6), (2, 1), (8, 7), (2, 2), (8, 8), (2, 3), (8, 9), (2, 4), (8, 10), (2, 5), (8, 11), (3, 0), (9, 6), (3, 1), (9, 7), (3, 2), (9, 8), (3, 3), (9, 9), (3, 4), (9, 10), (3, 5), (9, 11), (4, 0), (10, 6), (4, 1), (10, 7), (4, 2), (10, 8), (4, 3), (10, 9), (4, 4), (10, 10), (4, 5), (10, 11), (5, 0), (11, 6), (5, 1), (11, 7), (5, 2), (11, 8), (5, 3), (11, 9), (5, 4), (11, 10), (5, 5), (11, 11)])
dict_keys([(0, 0), (6, 6), (0, 1), (6, 7), (0, 2), (6, 8), (0, 3), (6, 9), (0, 4), (6, 10), (0, 5), (6, 11), (1, 0), (7, 6), (1, 1), (7, 7), (1, 2), (7, 8), (1, 3), (7, 9), (1, 4), (7, 10), (1, 5), (7, 11), (2, 0), (8, 6), (2, 1), (8, 7), (2, 2), (8, 8), (2, 3), (8, 9), (2, 4), (8, 10), (2, 5), (8, 11), (3, 0), (9, 6), (3, 1), (9, 7), (3, 2), (9, 8), (3, 3), (9, 9), (3, 4), (9, 10), (3

Correction for the (n-n)--(p-p) interaction

In [37]:




tot_hamiltonian=hamiltonian_qq
values,psi=eigsh(hamiltonian_qq,k=1)
e=values[0]
approximations=[]
nsteps_iteration=30
single_term = hamiltonian_rq  # Start with initial term
for i in trange(nsteps_iteration):
    if i > 0:
        single_term = hamiltonian_rr @ single_term  # Efficient update
    approximations.append(hamiltonian_qr @ single_term)  # Store result


h_qq=0.
for key,value in one_body.items():
    idx_a,idx_b=key
    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

two_body_corrections_nn=[]
for operator in approximations:
    twobody_term={}
    for i,b in enumerate(QPC.quasiparticle_basis):
        for j,d in enumerate(QPC.quasiparticle_basis):
            idx_a,idx_b=np.nonzero(b)[0]
            idx_c,idx_d=np.nonzero(d)[0]

            twobody_term[(idx_a,idx_b,idx_c,idx_d)]=operator[i,j]#-external_potential_qq[i,j]
    two_body_corrections_nn.append(twobody_term)


 
history_errors_exact=[]
for i in trange(nsteps_iteration):
    
    tot_hamiltonian=hamiltonian_qq
    for j in range(i):
        tot_hamiltonian=tot_hamiltonian+approximations[j]/e**(j+1)
    values,psi=eigsh(tot_hamiltonian,k=1)
    e=values[0]
    print(e)
    print(np.abs((e-egs[0])/egs[0]),'index=',i)
    history_errors_exact.append(np.abs((e-egs[0])/egs[0]))

100%|██████████| 30/30 [00:00<00:00, 276.92it/s]

-23.424939176438905
0.021281327927614318 index= 0
-23.660573385004376
0.011436282099961281 index= 1
-23.776440337705573
0.006595238576618887 index= 2
-23.84153619377299
0.003875465034077657 index= 3
-23.879346892002793
0.002295694171798298 index= 4
-23.901575778911486
0.0013669478253059684 index= 5
-23.914738003161137
0.0008170162100258504 index= 6
-23.922572303628062
0.000489690871340785 index= 7
-23.927253517125507
0.00029410483274329186 index= 8
-23.93005873320525
0.0001768999454322681 index= 9
-23.931743299694087
0.00010651698031316644 index= 10
-23.932756446133823
6.41866531928286e-05 index= 11
-23.93336645087826
3.870001141599768e-05 index= 12
-23.933734017976484
2.3342670269412318e-05 index= 13
-23.93395562616363
1.4083646414071356e-05 index= 14
-23.934089289318713
8.499058780255482e-06 index= 15
-23.93416993206248
5.129719867634471e-06 index= 16
-23.934218596509602
3.096467925992865e-06 index= 17
-23.934247968041717
1.8692943273033239e-06 index= 18
-23.934265697372275
1.1285441

#### The $^{24}$ Mg case

In [ ]:
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=6
nparticles_b=6

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{28}$Si'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis)) 
        
        
        



size= 24 12
[-35.26713604]
total_m= [-1.21430643e-17]


Build the corresponding $H_{qq}$ using qubits

In [39]:
QPC=QuasiParticlesConverterOnlynnpp()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)



check the $H_{QQ}$

In [40]:
hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T

value,_=eigsh(hamiltonian_qq,k=1)

print(value[0])
print((value[0]-egs)/egs)

-34.64777143418115
[-0.01756209]


(if you want) adding the tree level correction (diagonal)

In [41]:
#### twobody terms
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

value,_=eigsh(h_qq,k=1)

print(value[0])
h_qq_1=h_qq.copy()

# four_body={}
# for i,b in enumerate(QPC.quasiparticle_basis):
#         idx_a1,idx_a2,idx_a3,idx_a4=np.nonzero(b)[0]
#         four_body[(idx_a1,idx_a2,idx_a3,idx_a4)]=hamiltonian_qq[i,i]-h_qq[i,i]
        
# print(four_body)
# h_qq=0.

# for key,value in one_body.items():
#     idx_a,idx_b=key

#     h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

# for key,value in twobody_nn.items():
#     idx_a,idx_b,idx_c,idx_d=key

#     h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
# for key,value in twobody_nnpp.items():
#     idx_a,idx_b,idx_c,idx_d=key

#     h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

# # for key,value in four_body.items():
# #     idx_a,idx_b,idx_c,idx_d=key
# #     h_qq+=value*QubitBasis.four_body_matrix(idx_a,idx_b,idx_c,idx_d,idx_a,idx_b,idx_c,idx_d)


# value,_=eigsh(h_qq,k=1)
# print(h_qq-h_qq_1)
# print(value)

dict_keys([(0, 0), (6, 6), (0, 1), (6, 7), (0, 2), (6, 8), (0, 3), (6, 9), (0, 4), (6, 10), (0, 5), (6, 11), (1, 0), (7, 6), (1, 1), (7, 7), (1, 2), (7, 8), (1, 3), (7, 9), (1, 4), (7, 10), (1, 5), (7, 11), (2, 0), (8, 6), (2, 1), (8, 7), (2, 2), (8, 8), (2, 3), (8, 9), (2, 4), (8, 10), (2, 5), (8, 11), (3, 0), (9, 6), (3, 1), (9, 7), (3, 2), (9, 8), (3, 3), (9, 9), (3, 4), (9, 10), (3, 5), (9, 11), (4, 0), (10, 6), (4, 1), (10, 7), (4, 2), (10, 8), (4, 3), (10, 9), (4, 4), (10, 10), (4, 5), (10, 11), (5, 0), (11, 6), (5, 1), (11, 7), (5, 2), (11, 8), (5, 3), (11, 9), (5, 4), (11, 10), (5, 5), (11, 11)])
-34.64777143418111


In [42]:
for key,value in twobody_nn.items():
    a,b,c,d=key
    if a<6 and b<6 and c<6 and d<6:
        print(a,b,c,d,'\n')

print(twobody_nn.keys())

0 1 0 1 

0 2 0 2 

0 3 0 3 

0 4 0 4 

0 5 0 5 

1 2 1 2 

1 3 1 3 

1 4 1 4 

1 5 1 5 

2 3 2 3 

2 4 2 4 

2 5 2 5 

3 4 3 4 

3 5 3 5 

4 5 4 5 

dict_keys([(0, 1, 0, 1), (6, 7, 6, 7), (0, 2, 0, 2), (6, 8, 6, 8), (0, 3, 0, 3), (6, 9, 6, 9), (0, 4, 0, 4), (6, 10, 6, 10), (0, 5, 0, 5), (6, 11, 6, 11), (1, 2, 1, 2), (7, 8, 7, 8), (1, 3, 1, 3), (7, 9, 7, 9), (1, 4, 1, 4), (7, 10, 7, 10), (1, 5, 1, 5), (7, 11, 7, 11), (2, 3, 2, 3), (8, 9, 8, 9), (2, 4, 2, 4), (8, 10, 8, 10), (2, 5, 2, 5), (8, 11, 8, 11), (3, 4, 3, 4), (9, 10, 9, 10), (3, 5, 3, 5), (9, 11, 9, 11), (4, 5, 4, 5), (10, 11, 10, 11)])


First order

In [24]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.

for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

# for key,value in four_body.items():
#     idx_a,idx_b,idx_c,idx_d=key
#     h_qq+=value*QubitBasis.four_body_matrix(idx_a,idx_b,idx_c,idx_d,idx_a,idx_b,idx_c,idx_d)
value,_=eigsh(h_qq,k=1)
print(h_qq)
print(value[0])

    


  (0, 0)	-32.979600000000005
  (0, 1)	-0.900781889989654
  (0, 2)	-0.7347060355029268
  (0, 3)	1.7984746068253425
  (0, 4)	0.900781889989654
  (0, 5)	1.1336192497488327
  (0, 6)	-1.3995613925794368
  (0, 10)	-0.900781889989654
  (0, 11)	-1.9314456782406446
  (0, 12)	0.6017349640876247
  (1, 0)	-0.900781889989654
  (1, 1)	-28.18783333333334
  (1, 2)	-0.7177133829043456
  (1, 3)	0.7177133829043456
  (1, 4)	0.8598999999999998
  (1, 7)	1.1336192497488327
  (1, 8)	-1.3995613925794368
  (1, 10)	-0.4062999999999996
  (1, 13)	-1.9314456782406446
  (1, 14)	0.6017349640876247
  (2, 0)	-0.7347060355029268
  (2, 1)	-0.7177133829043456
  (2, 2)	-19.96362285714286
  (2, 3)	0.9008999999999998
  (2, 5)	0.8598999999999998
  (2, 7)	0.900781889989654
  (2, 9)	-1.3995613925794368
  (2, 11)	-0.4062999999999996
  (2, 13)	-0.900781889989654
  (2, 15)	0.6017349640876247
  (3, 0)	1.7984746068253425
  (3, 1)	0.7177133829043456
  (3, 2)	0.9008999999999998
  (3, 3)	-18.15384380952381
  (3, 6)	0.8598999999999998
 

Arbitrary order with a loop

In [ ]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
print(external_potential_qq.shape)

h_qq=0.

for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
    
for r,correction in enumerate(two_body_corrections):

    delta_hamiltonian=0.    
    for j in trange(r):
        for keys,values in two_body_corrections[j].items():
            idx_a,idx_b,idx_c,idx_d=keys
            correction_term=values*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)/((e)**(j+1))
            delta_hamiltonian+=correction_term
    for j in trange(r):
        for keys,values in two_body_corrections_nn[j].items():
            idx_a,idx_b,idx_c,idx_d=keys
            correction_term=values*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)/((e)**(j+1))
            delta_hamiltonian+=correction_term
                    
    value,_=eigsh(h_qq+delta_hamiltonian,k=1)
    e=value[0]
    print('iteration=',r,' energy=',e,'\n')



    

(400, 400)


0it [00:00, ?it/s]


iteration= 0  energy= -140.91443599062347 



100%|██████████| 1/1 [00:17<00:00, 17.08s/it]


iteration= 1  energy= -143.04808162671566 



...

iteration= 3  energy= -143.43555032318528 



100%|██████████| 4/4 [01:16<00:00, 19.22s/it]


iteration= 4  energy= -143.44813738190797 



  0%|          | 0/5 [00:03<?, ?it/s]


KeyboardInterrupt: 

#### Other Cases

In [271]:
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=4

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{22}$Ne'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis)) 
        
        
        



size= 24 12
[-57.45408791]
total_m= [4.32321381e-17]


Build the corresponding $H_{qq}$ using qubits

In [272]:
QPC=QuasiParticlesConverter()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)



check the $H_{QQ}$

In [273]:
hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T

value,_=eigsh(hamiltonian_qq,k=1)

print(value[0])

-51.352457221312235


In [274]:
print(hamiltonian_qq)

  (0, 0)	-48.05152222222223
  (0, 1)	0.8598999999999996
  (0, 2)	0.9007818899896537
  (0, 3)	1.133619249748832
  (0, 4)	-1.3995613925794361
  (0, 5)	-0.4062999999999996
  (0, 6)	-0.9007818899896537
  (0, 7)	-1.9314456782406437
  (0, 8)	0.6017349640876244
  (0, 15)	0.6897999999999996
  (0, 30)	-0.4062999999999996
  (0, 45)	-0.900781889989654
  (0, 60)	-1.9314456782406446
  (0, 75)	0.6017349640876247
  (1, 0)	0.8598999999999996
  (1, 1)	-46.40699047619047
  (1, 2)	-0.9007818899896537
  (1, 3)	-0.7347060355029265
  (1, 4)	1.7984746068253417
  (1, 5)	0.6897999999999995
  (1, 9)	-0.9007818899896537
  (1, 10)	-1.9314456782406437
  (1, 11)	0.6017349640876244
  (1, 16)	0.6897999999999996
  (1, 31)	-0.4062999999999996
  :	:
  (88, 58)	0.7177133829043456
  (88, 73)	0.9008999999999998
  (88, 77)	0.6017349640876244
  (88, 79)	-0.9007818899896537
  (88, 81)	-1.3995613925794361
  (88, 83)	0.9007818899896537
  (88, 84)	1.7984746068253417
  (88, 86)	-0.9007818899896537
  (88, 87)	0.9008999999999994
  

(if you want) adding the tree level correction (diagonal)

First order

In [275]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.

for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

# for key,value in four_body.items():
#     idx_a,idx_b,idx_c,idx_d=key
#     h_qq+=value*QubitBasis.four_body_matrix(idx_a,idx_b,idx_c,idx_d,idx_a,idx_b,idx_c,idx_d)
value,_=eigsh(h_qq,k=1)
print(h_qq)
print(value[0])

    


  (0, 0)	-32.348722222222236
  (0, 1)	0.8598999999999996
  (0, 2)	0.9007818899896537
  (0, 3)	1.133619249748832
  (0, 4)	-1.3995613925794361
  (0, 5)	-0.4062999999999996
  (0, 6)	-0.9007818899896537
  (0, 7)	-1.9314456782406437
  (0, 8)	0.6017349640876244
  (0, 15)	0.6897999999999996
  (0, 30)	-0.4062999999999996
  (0, 45)	-0.900781889989654
  (0, 60)	-1.9314456782406446
  (0, 75)	0.6017349640876247
  (1, 0)	0.8598999999999996
  (1, 1)	-30.704190476190497
  (1, 2)	-0.9007818899896537
  (1, 3)	-0.7347060355029265
  (1, 4)	1.7984746068253417
  (1, 5)	0.6897999999999995
  (1, 9)	-0.9007818899896537
  (1, 10)	-1.9314456782406437
  (1, 11)	0.6017349640876244
  (1, 16)	0.6897999999999996
  (1, 31)	-0.4062999999999996
  (1, 46)	-0.900781889989654
  (1, 61)	-1.9314456782406446
  (1, 76)	0.6017349640876247
  (2, 0)	0.9007818899896537
  (2, 1)	-0.9007818899896537
  (2, 2)	-29.156129365079376
  (2, 3)	-0.7177133829043454
  (2, 4)	0.7177133829043454
  (2, 6)	0.6897999999999995
  (2, 9)	-0.40629999

Second order

In [ ]:
h_qq=0.
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        
        h_qq+=one_body[i,j]*QubitBasis.adag_a_matrix(i,j)


for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                h_qq+=(twobody_nnpp[i,j,k,l]+twobody_nn[i,j,k,l]+two_body_corrections[0][i,j,k,l]/value[0])*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                
value,_=eigsh(h_qq+external_potential_qq,k=1)

print(value)

[-53.5305729]


Third order

In [ ]:
h_qq=0.
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        
        h_qq+=one_body[i,j]*QubitBasis.adag_a_matrix(i,j)


for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                h_qq+=(twobody_nnpp[i,j,k,l]+twobody_nn[i,j,k,l]+two_body_corrections[0][i,j,k,l]/value[0]+two_body_corrections[1][i,j,k,l]/value[0]**2)*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                
value,_=eigsh(h_qq+external_potential_qq,k=1)

print(value)

[-54.25552219]


Arbitrary order with a loop

In [ ]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
print(external_potential_qq.shape)

hamiltonian_qq_qubit=0. 

for i in range(len(QPC.couples)): #just neutrons
        for j in range(len(QPC.couples)):
            
            hamiltonian_qq_qubit+=one_body[i,j]*QubitBasis.adag_a_matrix(i,j)
            
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                hamiltonian_qq_qubit+=(twobody_nnpp[i,j,k,l]+twobody_nn[i,j,k,l])*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                #hamiltonian_qq_qubit+=four_body[i,j,k,l]*QubitBasis.four_body_matrix(i,j,k,l,i,j,k,l)

for r,correction in enumerate(two_body_corrections[:5]):

        
    overall_twobody=np.zeros((len(QPC.couples),len(QPC.couples),len(QPC.couples),len(QPC.couples)))
    for j in range(r):
        overall_twobody+=(two_body_corrections[j])/((e)**(j+1))
    delta_hamiltonian=0.
    for i in range(len(QPC.couples)): #just neutrons
        for j in range(len(QPC.couples)):
            for k in range(len(QPC.couples)):
                for l in range(len(QPC.couples)):
                    #print(i,j,k,l)
                    #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                    delta_hamiltonian+=overall_twobody[i,j,k,l]*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                    
    value,_=eigsh(hamiltonian_qq_qubit+delta_hamiltonian,k=1)
    e=value[0]
    print('iteration=',r,' energy=',e,'\n')

delta_fourbody=0.
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                delta_fourbody+=four_body[i,j,k,l]*QubitBasis.four_body_matrix(i,j,k,l,i,j,k,l)
value,_=eigsh(hamiltonian_qq_qubit+delta_hamiltonian+delta_fourbody,k=1)
e=value[0]
print('iteration=',r,' energy=',e,'\n')

    

(225, 225)
iteration= 0  energy= -66.3824753272453 

iteration= 1  energy= -69.75675976421405 

iteration= 2  energy= -70.89029762179018 

iteration= 3  energy= -71.34028995152087 

iteration= 4  energy= -71.53242775338654 

iteration= 4  energy= -79.1134283333448 

